# Social Cops Assignment - Agriculture Commodities, Prices & Seasons
<br>
We'll start by including our required libraries


In [1]:
library(ggseas)
library(zoo)
library(tseries)
library(forecast)
library(data.table)

Warning message:
"package 'ggseas' was built under R version 3.5.1"Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.5.1"Warning message:
"package 'zoo' was built under R version 3.5.1"
Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric

Warning message:
"package 'tseries' was built under R version 3.5.1"Warning message:
"package 'forecast' was built under R version 3.5.1"Warning message:
"package 'data.table' was built under R version 3.5.1"

Read Data and format data

In the `date` attribute, the data only has year and month. To perform a time series analysis it has been converted to R's date format.

In [2]:
price_data<-fread("D:\\Python\\SocialCops\\Monthly_data_cmo.csv")
msp_data<-fread("D:\\Python\\SocialCops\\CMO_MSP_Mandi.csv")
price_data<-as.data.table(price_data)


price_data$date<-as.Date(paste(price_data$date,"-01",sep=""))

#### Preprocessing 
<br>
I assumed that a modal price of 0 doesn't make much sense, hence I decided to remove all such instances so as to perform a clean analysis. 

Furthermore, I have defined a function `remove_outliers` which removes outliers in the data by looking at the inter-quantile range. Any value which crosses 1.5 times the IQR on either side is termed as an outlier and removed from the data. 

For seasonal analysis I have resorted to using only `modal_price` attribute as it represents the data best corresponding to every month. *I also investigate whether it is dependent on quantity arrived.* All attributes which I do not use are hence discarded. 

In [3]:
remove_outliers <- function(x, na.rm = TRUE, ...) {
  qnt <- quantile(x, probs=c(.25, .75), na.rm = na.rm, ...)
  H <- 1.5 * IQR(x, na.rm = na.rm)
  y <- x
  y[x < (qnt[1] - H)] <- NA
  y[x > (qnt[2] + H)] <- NA
  y
}

In [4]:
# remove 0 modal_price data points
price_data<-price_data[price_data$modal_price!=0]
# remove outliers
price_data[, new:=modal_price][, new:=remove_outliers(modal_price), by=.(APMC, Commodity)]
price_data<-na.omit(price_data)
# Find correlation of price to supply
cor(price_data$modal_price, price_data$arrivals_in_qtl)

[1] -0.03976585

Since there is an extremely weak correlation of approximately -0.04 , we can safely remove the arrivals_in_qtl from our data and simply concentrate on modal price.

In [5]:
# remove extra attributes
price_data[,c('Month','arrivals_in_qtl','min_price','max_price','district_name','state_name')]<-NULL

#### Finding Seasonality Type and Deseasonalising Prices
<br>
My algorithm to address the above question is as follows:
<br> <br>
1. Identify and separate **trend** values from the data <br>
2 Since we don't know which type of seasonality the different APMC-Commodity clusters exhibit, I detrend according to both <br>
3. Now to get the season variation, I took mean of data in every cluster according to the quarters in every year <br>
4. To see the influence of any temporal pattern, I use the autocorrelation function: `acf()` provided by R and for each APC-Commodity cluster residuals and calculate it's sum. I pass both additive and multiplicative residuals in order to compare their validity <br>
5. With the `compare_ssacf()` function I compare and assign in the dataset the corresponding seasonality type <br>

In [6]:
# Add trend
price_data$trend<-ave(price_data$modal_price, price_data[,1:2], FUN = function(x) rollmean(x, k=3, fill=x))
price_data<-na.omit(price_data)
head(price_data)
# De-trend data
price_data[,`:=`( detrended_a = modal_price - trend,  detrended_m = modal_price / trend )]
head(price_data)

# Make seasonal component
price_data[,`:=`(seasonal_a = mean(detrended_a, na.rm = TRUE),
             seasonal_m = mean(detrended_m, na.rm = TRUE)), 
          by=.(APMC,Commodity,quarter(date)) ]
head(price_data)
                      
#print(price_data[is.na(price_data$seasonal_m)])
# Make residual
price_data[,`:=`( residual_a = detrended_a - seasonal_a, 
                 residual_m = detrended_m / seasonal_m )]
head(price_data)
                     
price_data<-as.data.table(na.omit(price_data))
# Auto-correlated factor
ssacf<- function(x) sum(acf(x, na.action = na.remove, plot= FALSE)$acf^2)
compare_ssacf<-function(add,mult) ifelse(ssacf(add)< ssacf(mult), 
                                         "Additive", "Multiplicative") 
                     
price_data[, `:=` (seasType = mapply(compare_ssacf, residual_a, residual_m)), by = .(APMC, Commodity)]

APMC,Commodity,Year,modal_price,date,new,trend
Ahmednagar,Bajri,2015,1463,2015-04-01,1463,1463.000
Ahmednagar,Bajri,2016,1875,2016-04-01,1875,1651.000
Ahmednagar,Wheat(Husked),2015,1731,2015-04-01,1731,1731.000
Ahmednagar,Wheat(Husked),2016,1999,2016-04-01,1999,1776.667
Ahmednagar,Sorgum(Jawar),2015,1900,2015-04-01,1900,1900.000
Ahmednagar,Sorgum(Jawar),2016,2119,2016-04-01,2119,1978.000


APMC,Commodity,Year,modal_price,date,new,trend,detrended_a,detrended_m
Ahmednagar,Bajri,2015,1463,2015-04-01,1463,1463.000,0.0000,1.000000
Ahmednagar,Bajri,2016,1875,2016-04-01,1875,1651.000,224.0000,1.135675
Ahmednagar,Wheat(Husked),2015,1731,2015-04-01,1731,1731.000,0.0000,1.000000
Ahmednagar,Wheat(Husked),2016,1999,2016-04-01,1999,1776.667,222.3333,1.125141
Ahmednagar,Sorgum(Jawar),2015,1900,2015-04-01,1900,1900.000,0.0000,1.000000
Ahmednagar,Sorgum(Jawar),2016,2119,2016-04-01,2119,1978.000,141.0000,1.071284


APMC,Commodity,Year,modal_price,date,new,trend,detrended_a,detrended_m,seasonal_a,seasonal_m
Ahmednagar,Bajri,2015,1463,2015-04-01,1463,1463.000,0.0000,1.000000,88.466667,1.052843
Ahmednagar,Bajri,2016,1875,2016-04-01,1875,1651.000,224.0000,1.135675,88.466667,1.052843
Ahmednagar,Wheat(Husked),2015,1731,2015-04-01,1731,1731.000,0.0000,1.000000,85.833333,1.047333
Ahmednagar,Wheat(Husked),2016,1999,2016-04-01,1999,1776.667,222.3333,1.125141,85.833333,1.047333
Ahmednagar,Sorgum(Jawar),2015,1900,2015-04-01,1900,1900.000,0.0000,1.000000,9.066667,1.005833
Ahmednagar,Sorgum(Jawar),2016,2119,2016-04-01,2119,1978.000,141.0000,1.071284,9.066667,1.005833


APMC,Commodity,Year,modal_price,date,new,trend,detrended_a,detrended_m,seasonal_a,seasonal_m,residual_a,residual_m
Ahmednagar,Bajri,2015,1463,2015-04-01,1463,1463.000,0.0000,1.000000,88.466667,1.052843,-88.466667,0.9498097
Ahmednagar,Bajri,2016,1875,2016-04-01,1875,1651.000,224.0000,1.135675,88.466667,1.052843,135.533333,1.0786754
Ahmednagar,Wheat(Husked),2015,1731,2015-04-01,1731,1731.000,0.0000,1.000000,85.833333,1.047333,-85.833333,0.9548058
Ahmednagar,Wheat(Husked),2016,1999,2016-04-01,1999,1776.667,222.3333,1.125141,85.833333,1.047333,136.500000,1.0742908
Ahmednagar,Sorgum(Jawar),2015,1900,2015-04-01,1900,1900.000,0.0000,1.000000,9.066667,1.005833,-9.066667,0.9942011
Ahmednagar,Sorgum(Jawar),2016,2119,2016-04-01,2119,1978.000,141.0000,1.071284,9.066667,1.005833,131.933333,1.0650718


Now to deseasonalize, we follow the simple rule:
<br>
1. In case of 'Additive', subtract the seasonal component from the original data and whatever is left is the deseasonalized price<br>

2. Similarly, in case of 'Multiplcative', divide from the original data by the seasonal component.<br>

In [7]:
price_data$deSeasonalized<-0
price_data[price_data$seasType=='Multiplicative']$deSeasonalized<-price_data$modal_price/price_data$seasonal_m
price_data[price_data$seasType=='Additive']$deSeasonalized<-price_data$modal_price-price_data$seasonal_a

Finally the data table looks like

In [8]:
head(price_data)
write.csv(price_data, "FinalData.csv")

APMC,Commodity,Year,modal_price,date,new,trend,detrended_a,detrended_m,seasonal_a,seasonal_m,residual_a,residual_m,seasType,deSeasonalized
Ahmednagar,Bajri,2015,1463,2015-04-01,1463,1463.000,0.0000,1.000000,88.466667,1.052843,-88.466667,0.9498097,Multiplicative,1389.572
Ahmednagar,Bajri,2016,1875,2016-04-01,1875,1651.000,224.0000,1.135675,88.466667,1.052843,135.533333,1.0786754,Multiplicative,1780.893
Ahmednagar,Wheat(Husked),2015,1731,2015-04-01,1731,1731.000,0.0000,1.000000,85.833333,1.047333,-85.833333,0.9548058,Multiplicative,1652.769
Ahmednagar,Wheat(Husked),2016,1999,2016-04-01,1999,1776.667,222.3333,1.125141,85.833333,1.047333,136.500000,1.0742908,Multiplicative,1908.657
Ahmednagar,Sorgum(Jawar),2015,1900,2015-04-01,1900,1900.000,0.0000,1.000000,9.066667,1.005833,-9.066667,0.9942011,Multiplicative,1888.982
Ahmednagar,Sorgum(Jawar),2016,2119,2016-04-01,2119,1978.000,141.0000,1.071284,9.066667,1.005833,131.933333,1.0650718,Multiplicative,2106.712


#### Maximum Fluctuation APC-Commodity Clusters

<br>
For this problem, I found the standard deviation corresponding to all APMC-Commodity-Year groups first and then divided by the mean of the same group. I divide by mean because that would normalize our data. Otherwise we will not be able to compare two commodities with different price scales. 

In [9]:
var_price<-price_data[, sd(modal_price)/mean(modal_price) , by = .(APMC, Commodity, Year)]

In [10]:
var_price<-na.omit(var_price)
head(var_price)

APMC,Commodity,Year,V1
Ahmednagar,Bajri,2015,0.05777363
Ahmednagar,Bajri,2016,0.09594013
Ahmednagar,Wheat(Husked),2015,0.04692434
Ahmednagar,Wheat(Husked),2016,0.06013370
Ahmednagar,Sorgum(Jawar),2015,0.06648933
Ahmednagar,Sorgum(Jawar),2016,0.06317775


I order the table in order to show the most fluctuating commodities.

In [11]:
var_price<-var_price[order(-V1)]

In [12]:
head(var_price)
write.csv(var_price,"Fluct.csv")

APMC,Commodity,Year,V1
Nashik,Raddish,2015,1.821567
Ramtek,Jack Fruit,2015,1.676578
Pune-Pimpri,Coriander,2016,1.653172
Akluj,Coriander,2016,1.596218
Chandrapur,Orange,2016,1.410355
Ramtek,Jambhul,2015,1.376997


### MSP vs Deseasonalized prices - head to head
<br>

I created the matrix `mean_seasonal` below to get the mean of seasonal data of each commodity in every year to compare to the MSP from the other data. <br>
Just to be sure that the names aren't different in terms of only capitalization, I have converted names of both to lower case. <br>
Thereafter, I have simply merged the two datasets to get a head to head comparison.

In [13]:
mean_seasonal<-price_data[, mean(deSeasonalized) , by = .(Commodity, Year)]

In [14]:
msp_data$commodity<-tolower(msp_data$commodity)
msp_data$Commodity<-msp_data$commodity
msp_data$commodity<-NULL
msp_data$msp_filter<-NULL
msp_data$Year<-msp_data$year
msp_data$year<-NULL

mean_seasonal$Commodity<-tolower(mean_seasonal$Commodity)

In [15]:
comp_matrix<-merge(msp_data,mean_seasonal, by=c('Commodity', 'Year'))
comp_matrix<-unique(comp_matrix, by=c('Commodity','Year'))
colnames(comp_matrix)<-c('Commodity','Year','Type','msprice','Deseasonalized')

In [16]:
head(comp_matrix)
write.csv(comp_matrix,"CompMatrix.csv")

Commodity,Year,Type,msprice,Deseasonalized
bajri,2014,Kharif Crops,1250,1422.662
bajri,2015,Kharif Crops,1275,1405.908
bajri,2016,Kharif Crops,1330,1606.848
coconut,2014,Other Crops,1425,1282.121
coconut,2015,Other Crops,1500,1423.477
coconut,2016,Other Crops,1600,1071.919
